# 关于GPT中auto-debias的想法

## 零：偏见评估的数据集

- 评估偏见的数据集：

    这些数据集的收集过程，都是很值得考究的，需要意识到，如果哪一天需要真正收集数据构造数据集，需要灵感可以到这些论文中来找
    - SEAT  只能评估encoder模型
    
    - CrowS-Pairs   应该不可以用于decoder模型，这个模型的工作原理就是：
    
      提供stereotyped和anti-stereotyped的句子对，判断模型对句子对的反应是不是不一样的。
      
      其中有九种句子对，代表着九种偏见
      <img src="..\Apictures\10.png" alt="7" style="zoom:30%;" />
      
      🚩但是如何打分呢？根据不同任务对输入的句子打分？什么任务都可以?？不太理解。。。得读论文
       破案了，完全基于MLM的fill mask任务。具体为：
       
       句子对中有重叠的单词（偏见无关）和不重叠的单词（偏见相关）
       fill mask预测重叠的单词，每次mask掉一个重叠单词，预测一个概率，遍历所有重叠单词，
       叠加概率得到每一个句子的S得分
       （注意，偏见相关的单词不会mask掉！！）
       
       如果某一个句子有较高的S值，说明该句子出现的概率较高，也就是该句子中未重叠词出现的概率比较高。
       
       无偏见的模型应该保证句子对中的两个句子出现的概率一样，但是严格一样又是不太可能的，所以需要好多句子对，可能有的句子对stereotyped出现的概率比较高，有的句子对anti-stereotyped出现的概率比较高，但是只要保证总体上二者频率是一样的就可以。
       
      所以stereotyped具有较高出现可能性的频率为50%，那么效果就是最好的。
      
    - StereoSet
    
        待了解


## 一、记录不成熟的思路

    让模型生成大量文本
    
    使用偏见检测模型检测文本中是否存在偏见（如果可能的话，定位偏见）:这方面的技术了解不多，问下gpt有没有从文本中检测偏见的方法
    
    使用带有偏见的生成文本，制造反向提示，制造一个“平衡数据集”
    
    使用prompt tuning的方法，让微调参数代替诱导偏见的提示，使用流形学习让目标词在流行空间上更接近
    
    使用这个数据集中的文本，再对语言生成模型进行微调
    
    
    那么这篇文章的贡献就可以是：
    
    1.提出了一种在文本中检测偏见的方法。
    2.提出了一种针对语言生成模型/任务的auto-debias方法，填补了之前的auto-debias方法只能应用于fill mask任务的encoder模型的空白。
    3.causual debias论文中提出了，auto-debias模型在经过其它数据集微调后偏见可能会重新出现。这种方法有没有可能避免这种问题呢？
      需要做实验验证，如果试验表明可以，那么这就算一个贡献。

###  更具体的一个思路：
    其实偏见类型也就那几种：
        性别偏见  不同性别与不同的职业相关联
        种族偏见  对不同种族的积极/消极关联
        年龄偏见  对不同年龄的积极/消极形容
        文化偏见  对不同文化的积极/消极特征
    可以看到大多数都跟情感有关，要不就直接处理模型对不同群体的情感吧，性别偏见爱咋咋地吧。
    
    所以目标可以缩小为取出gpt中对性别/种族/年龄/文化的情感偏见
    
    一个思路：
    
        文本情感三分类模型是现成的
        
        找一个prompt模板放在开头，让gpt生成句子，对这个句子进行积极/中立/消极的情感三分类
        
        通过束搜索，找出使生成句子情感最偏激最不中立的prompt/使用连续的prompt learning找出让情感偏见最大的连续的prompt
        
        然后利用生成的prompt, 生成句子，对句子进行情感判断，目标是让情感尽量中立，所以损失可以定义为 "到中立的距离"，然后反向传播更新模型
     
     思路中的问题&注意点:
         
         🚩 prompt要怎么设计？什么样的结构？需要考虑 可以先读ADEPT找找灵感，再回来思考这个。
###  关于评估方法和初始化prompt的记录：
     🚩 gpt模型纠偏结束后，用什么指标来评估?
         好像现有的评估数据集都是针对fill masked的

         直接问gpt: 对于语言生成模型，有哪些数据集可以评估其中的偏见？

         回答如下图：
<img src="..\Apictures\9.png" alt="7" style="zoom:50%;" />


         其中的BOLD和GNLI好像靠谱一些，其它的到底能不能？🚩有时间再了解这几个其它的指标
         BOLD包含一系列的prompt来引导语言生成模型 生成有偏见的内容，但是对于生成的内容，没有方法可以量化其中的偏见
         🚩
         但是BOLD数据集还可以有其它作用，其中包含的prompt可以直接用，作为初始化的prompt（也可以作为continous prompt learning的显式token输入，然后再微调关于prompt的参数）---The real ToxicityPrompt dataset也可以

         其实auto-debias也没必要非得找让偏见最大化的prompt，直接自己定义一个prompt就可以，也可以实现去偏见的效果
         但是这样可能会疑惑，为什么要用这个prompt，而不是其它prompt呢?
         所以为了最大程度挖掘模型中的bias，就需要找一个这样子的prompt
         （注意，用来挖掘prompt的度量和最小化bias的损失函数其实是一样的
          所以，gpt用来挖掘是偏见最大化prompt的标准和最终的优化目标都可以是一样的
          都可以围绕Nbias来设计一个度量标准/损失函数）
          但是用来量化模型评估效果的方法就不能是这个了，因为你是用这个作为损失进行优化的，性能肯定好啊，得换一个，🚩换什么呢？）


         那么对于语言生成模型，还有必要挖掘这样一个prompt吗?
         不是必要的，但是能挖掘就更好。主要点就在于：
             🚩初始的prompt有了，要怎么改prompt呢？
         这个点解决不了，就不要想着挖掘了。  （先去读ADEPT的论文，看看能有什么idea）

             
           
             
             

### 关于半自动标注的思路：
    
        使用NER任务，专门定义一个Entity名叫BIAS，
        目标是得到一个能准确识别出BIAS实体的NER模型
        
        ① 然后先找人进行（或者自己课题组进行）Manual annotation，也就是手工标注
        
        ② 利用手工标注的数据集，初步训练一个BIAS识别模型
        
        ③ 利用训练好的模型进行人工的半自动标注，得到更多的数据
        
        ④ 利用更多的数据训练一个更好的模型，用于第三步，提升半自动标注的准确性和人工标注的复杂度
        
        ⑤ 第三四步循环进行，直到得到一个能准确识别BIAS的NER模型，同时也获得了一个足够大的数据集
    
    > 每个环节的难点
    
    ① 不管能不能标注出来，得有需要标注的句子，也就是语料库，才能开展标注的工作，选择哪些数据集来标注其实也是一门学问
        最好是选择偏见比较集中的领域的语料来进行标记，如果选择的预料中都没有偏见，
        一是浪费标记人的精力，
        二是最终产出的可以用来训练识别模型的（带有偏见标记的）可用数据太少，模型也就训练不好了。
    
    ② 有了“富含”偏见的数据集，还得有能够识别并标记出偏见的领域专业人员
        比如收集的数据集是医学领域的，法律领域，新闻领域的，那么必须是有经验的行业人员才能识别出来
        跟这些人的沟通也是非常重要的，必须要达成关于偏见“定义”的共识
    
    ③ 达成的共识以什么样的方式呈现也很重要：
        Nbias论文中有一个   可能暗示偏见的词表
        这些词表中的词语出现在句子中时，就表示可能有偏见了
<img src="..\Apictures\8.png" alt="7" style="zoom:50%;" />
        
    ④ 保证标注的可靠性
        如果每句话只有一个人标注，那么随机性太高了，如果这个人摸鱼乱标，那么数据质量不能得到保证
        
        标注阶段：
            同一份数据不能只有一个人标注，至少两人
            标注不一致的地方，需要拿出来商量，达成一致后重新加入数据集
        检查阶段：
            两个人判断标注 是否正确
            然后用kappa系数判断两人对标注结果的判断是否是一致的
            如果两个人都判断数据集标注质量较高，且kappa系数较高，
            那么就说明一致认为数据集质量较高，可以使用了。
    ⑤ Perpetuation bias tests for bias detection这里表现不好
        即使在中性的不带偏见的语境下，也容易把一些没见过的短语标记为BIAS ENTITY
        说明学到的偏见标识模型不能捕捉到中性的语境，只是看到了这些在偏见语境中经常出现的词语，
          就把这些词语标记了
         有效的模型应该是根据上下文做出判断的，而不是见到这个词就。。。
        

### 关于prompt灵感的记录：

    AutoPrompt: Eliciting Knowledge from Language Models with Automatically Generated Prompts（EMNLP2021）
    
    仔细体会这个题目的意思
    通过自动生成的prompt来挖掘LM中的知识
    （然后利用这些知识进行分类任务，实现上也就是找找prompt learning中的prompt来更准确的输出[mask]）
    括号里不是重点，重点是这个角度可以作为我们使用prompt来自动挖掘gpt模型中偏见的切入点。
    意思就是，"你看，auto prompt可以挖掘知识，自然就可以挖掘偏见"

## ADEPT灵感

- For the word tuples,

    we use neutral word lists employed in previous debiasing methods (Kaneko and Bollegala 2021; Caliskan, Bryson, and Narayanan 2017).
     For the binary gender setting, we use the pairwise attribute words from Zhao et al. (2018) and for the ternary religion setting, we use the attribute triplets from Liang et al. (2020).
    
    
    说明之前的传统去偏见的论文还是要读的，它们的方法虽然比较明确了，但是关于还有很多细节需要掌握
    比如这里，word tuples用什么呢??难道要自己定义吗?
    中性词tuple、成对的性别对立词（用于构建attribute tuples)、和宗教三元组就分别取自之前三个不同的论文
    读吧，时间紧迫。
- 还是要多读之前的论文，有很多启发：

    
    因为ADEPT这篇论文思路其实是从DPCE; (Kaneko and Bollegala 2021)演化出来的：
    - 设计中性词tuple和属性词tuple的思路
    - 以及使用二者embedding进行debias的思路
- ADEPT的思路还是只能用于基于编码器的BERT类模型，所以在gpt上使用prompt learning进行debias还是有可能的！
    
    - encoder可以直接把一个句子丢进去，然后在各个层得到每一个输入词的embedding，然后作者利用这些词的embedding求平均，做损失反向传播，才得以进行debias。
    
    
    - （依据现阶段对decoder的了解来看）
        decoder的输入，query只是最新生成的一个单词，
        key和value在masked attention是之前所有的生成，cross attention不知道（因为之前接触的是encoder-decoder都有的transformer，所以🚩gpt的具体细节还是的看的）
        反正就目前了解的gpt细节来看，ADEPT的工作还是不适合于gpt模型的！！



In [20]:
# 设置网络代理
import os

os.environ['http_proxy'] = "http://127.0.0.1:33210"
os.environ['https_proxy'] = "https://127.0.0.1:33210"
os.environ['all_proxy'] = "socks5://127.0.0.1:33211"

os.environ['http_proxy'], os.environ['all_proxy']

('http://127.0.0.1:33210', 'socks5://127.0.0.1:33211')

In [21]:
import torch
from torch import nn
from transformers import BertTokenizer, BertModel
from torchcrf import CRF

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')

SSLError: (MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /bert-base-uncased/resolve/main/vocab.txt (Caused by SSLError(SSLZeroReturnError(6, 'TLS/SSL connection has been closed (EOF) (_ssl.c:1131)')))"), '(Request ID: c1967375-dc90-46e4-9d34-338cd59bb0c6)')

In [5]:
import spacy

# 加载预训练的模型
nlp = spacy.load("en_core_web_sm")

# 处理文本
text = "Google was founded by Larry Page and Sergey Brin. China I love you"
doc = nlp(text)

# 打印识别的实体
for ent in doc.ents:
    print(ent.text, ent.label_)

# 打印每一个词的词性
for token in doc:
    print(token.text, token.pos_)

Google ORG
Larry Page PERSON
Sergey Brin PERSON
China GPE
Google PROPN
was AUX
founded VERB
by ADP
Larry PROPN
Page PROPN
and CCONJ
Sergey PROPN
Brin PROPN
. PUNCT
China PROPN
I PRON
love VERB
you PRON


In [ ]:
class GPT2LMHeadModel(GPT2PreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.transformer = GPT2Model(config)

        # 将 self.transformer 中最后一个 Block 层输出的隐藏状态的
        # 最后一个维度由 config.n_embd 投影为 config.vocab_size
        self.lm_head = nn.Linear(config.n_embd, config.vocab_size, bias=False)
    
    def forward(self, 参数见上表):
        # 是否以字典形式返回结果，默认为 True
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        transformer_outputs = self.transformer(
            input_ids,
            past_key_values=past_key_values,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            encoder_hidden_states=encoder_hidden_states,
            encoder_attention_mask=encoder_attention_mask,
            use_cache=use_cache,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )

        # [batch, sequence_len, embeded_dim]
        hidden_states = transformer_outputs[0]

        # [batch, sequence_len, embeded_dim] -> [batch, sequence_len, vocab_size]
        lm_logits = self.lm_head(hidden_states)

        # 计算损失
        loss = None
        if labels is not None:
            # [batch_size, sequence_len]
            labels = labels.to(lm_logits.device)

            # 最后一个 token 是本次计算预测的结果，计算损失时要去除
            # [batch, sequence_len - 1, vocab_size]
            shift_logits = lm_logits[..., :-1, :].contiguous()
            
            # 第一个 token 是 [CLS]，计算损失时要去除
            # [batch, sequence_len - 1]
            shift_labels = labels[..., 1:].contiguous()

            # 计算交叉熵损失
            loss_fct = CrossEntropyLoss()
            loss = loss_fct(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))

        return CausalLMOutputWithCrossAttentions(
            loss=loss,
            logits=lm_logits,
            past_key_values=transformer_outputs.past_key_values,
            hidden_states=transformer_outputs.hidden_states,
            attentions=transformer_outputs.attentions,
            cross_attentions=transformer_outputs.cross_attentions,
        )

In [ ]:
class GPT2Model(GPT2PreTrainedModel):
    def __init__(self, config):
        super().__init__(config)

        # token 的编码维度，默认 768
        self.embed_dim = config.hidden_size

        # 词向量编码
        self.wte = nn.Embedding(config.vocab_size, self.embed_dim)

        # token 的位置编码
        self.wpe = nn.Embedding(config.max_position_embeddings, self.embed_dim)

        self.drop = nn.Dropout(config.embd_pdrop)

        # Block 列表默认 12
        self.h = nn.ModuleList([GPT2Block(config, layer_idx=i) for i in range(config.num_hidden_layers)])

        # 层归一化层
        self.ln_f = nn.LayerNorm(self.embed_dim, eps=config.layer_norm_epsilon)

    def forward(self, 主要参数见 GTP2LMHeadModel):

         # 默认 False，是否需要输出 GPT2Model 处理过程中每个 block 的 attentions
        output_attentions = output_attentions if output_attentions is not None else self.config.output_attentions

        # 默认 False，是否需要输出 GPT2Model 中每个 GPT2Block 输出的 hidden states
        output_hidden_states = (
            output_hidden_states if output_hidden_states is not None else self.config.output_hidden_states
        )

        # 默认 True，输出 past_key_values
        use_cache = use_cache if use_cache is not None else self.config.use_cache

        if token_type_ids is not None:
            # [batch_size, sequence_len]
            token_type_ids = token_type_ids.view(-1, input_shape[-1])

         # 默认为 None
        if position_ids is not None:
            position_ids = position_ids.view(-1, input_shape[-1])

        # 默认为 None
        if past_key_values is None:
            past_length = 0
            # 生成 None 列表
            past_key_values = tuple([None] * len(self.h))
        else:
            past_length = past_key_values[0][0].size(-2)

        # 默认为 None
        if position_ids is None:
            # 生成绝对位置编码
            # [sequence_len]
            position_ids = torch.arange(past_length, input_shape[-1] + past_length, dtype=torch.long, device=device)
            # [1, sequence_len]
            position_ids = position_ids.unsqueeze(0).view(-1, input_shape[-1])

        if attention_mask is not None:
            # [batch, sequence_len]
            attention_mask = attention_mask.view(batch_size, -1)

            # [batch_size, 1, 1, seq_length]
            attention_mask = attention_mask[:, None, None, :]

            attention_mask = attention_mask.to(dtype=self.dtype) 

            # 将 Mask 位置设置为极小值，便于后续计算注意力分数
            # 形如：[[[[-0.0000e+00, -0.0000e+00, -0.0000e+00, -0.0000e+00, -0.0000e+00,
            #    -3.4028e+38, -3.4028e+38]]]]
            attention_mask = (1.0 - attention_mask) * torch.finfo(self.dtype).min

        # 词向量编码
        if inputs_embeds is None:
            inputs_embeds = self.wte(input_ids)

        # 计算位置编码
        position_embeds = self.wpe(position_ids)

        hidden_states = inputs_embeds + position_embed

        # 正则化
        hidden_states = self.drop(hidden_states)

        # input_shape: [batch, sequence_len] -> [batch, sequence_len, embeded_dim]
        output_shape = input_shape + (hidden_states.size(-1),)

        # 存储 past_key_values
        presents = () if use_cache else None

        # 分层计算
        for i, (block, layer_past) in enumerate(zip(self.h, past_key_values)):
            if self.model_parallel:
               # 调试代码时，未开启并行计算
               ......
            else:
                # (hidden_states, present, (attentions, ))
                outputs = block(
                    hidden_states,
                    layer_past=layer_past,
                    attention_mask=attention_mask,
                    head_mask=head_mask[i],
                    encoder_hidden_states=encoder_hidden_states,
                    encoder_attention_mask=encoder_attention_mask,
                    use_cache=use_cache,
                    output_attentions=output_attentions,
                )

            # 当前 Block 输出的结果
            hidden_states = outputs[0]

            if use_cache is True:
                # outputs[1]：(key, value)
                # past_key_values
                presents = presents + (outputs[1],)

        # 层归一化
        hidden_states = self.ln_f(hidden_states)

        hidden_states = hidden_states.view(output_shape)
    
        return BaseModelOutputWithPastAndCrossAttentions(
            last_hidden_state=hidden_states,
            past_key_values=presents,
            hidden_states=all_hidden_states,
            attentions=all_self_attentions,
            cross_attentions=all_cross_attentions,
        )